# Linux Kernel Module Parameters

The Linux kernel provides a simple framework, enabling drivers to declare parameters that the user can specify on either boot or module load and then have these parameters exposed in your driver as global variables.These module parameters also show up in `sysfs` system.

## 1. Loading a module with parameters
We're going the write a demo kernel module, trying to pass parameters to the module during loading.
```sh
insmod ./hello-parameters.ko irq=20 name=mydev debug=1
```
We start with the following 'Hello, World!' source file:

In [1]:
%%writefile hello-parameters.c
#include <linux/init.h>
#include <linux/module.h>
#include <linux/kernel.h>
#include <linux/moduleparam.h>

MODULE_LICENSE("GPL");
MODULE_AUTHOR("Clymber Loong");
MODULE_DESCRIPTION("A Hello, World Module");

static int   irq   = 10;
static int   debug = 0;
static char *name  = "Hello World"; 

static int hello_init(void)
{
    printk(KERN_ALERT "Hello, World! name=%s, irq=%d, debug=%d\n", name, irq, debug);
    return 0;
}

static void hello_exit(void)
{
    printk(KERN_ALERT "Bye Bye, World!\n");
}

module_init(hello_init);
module_exit(hello_exit);

Overwriting hello-parameters.c


### 1.1 Defining parameters

Defining a module parameter is done via `the macro module_param()`:
```c
// @name: both the parameter exposed to the user and the variable holding
//        the parameter inside your module;
// @type: holds the parameter’s data type;
// @perm: specifies the permissions of the corresponding file in sysfs.
module_param(name, type, perm);
```
The `type` argument holds the parameter’s data type; it is one of byte, short, ushort, int, uint, long, ulong, charp, bool, or invbool.  
These types are, respectively, a byte, a short integer, an unsigned short integer, an integer, an unsigned integer, a long integer, an unsigned long integer, a pointer to a char, a Boolean, and a Boolean whose value is inverted from what the user specifies.The byte type is stored in a single char and the Boolean types are stored in variables of type int.

In [2]:
%%writefile -a hello-parameters.c
module_param(irq, int, 0660);
module_param(debug, int, 0660);
module_param(name, charp, 0660);

Appending to hello-parameters.c


### 1.2 Documenting the parameters (Optional)

We can document your parameters by using `MODULE_PARM_DESC()`:

In [3]:
%%writefile -a hello-parameters.c

MODULE_PARM_DESC(irq, "The interrupt line number.");
MODULE_PARM_DESC(debug, "Debug mode, 1 is on, 0 is off.");
MODULE_PARM_DESC(name, "Your name");

Appending to hello-parameters.c


### 1.2 Passing parameters during loading module

In [4]:
! make

make -C /lib/modules/6.1.28/build M=/home/clymber/Studio/kerdevel/module_param modules
make[1]: Entering directory '/home/clymber/Studio/@linux-6.1.28'
  CC [M]  /home/clymber/Studio/kerdevel/module_param/hello-parameters.o
  CC [M]  /home/clymber/Studio/kerdevel/module_param/hello-rwv-param.o
  LD [M]  /home/clymber/Studio/kerdevel/module_param/module_param.o
  MODPOST /home/clymber/Studio/kerdevel/module_param/Module.symvers
  CC [M]  /home/clymber/Studio/kerdevel/module_param/module_param.mod.o
  LD [M]  /home/clymber/Studio/kerdevel/module_param/module_param.ko
make[1]: Leaving directory '/home/clymber/Studio/@linux-6.1.28'


### 1.3 List the parameters that the module supports

We can list the information about the module we built with command `modinfo`, including descriptions of its parameters:

In [5]:
! modinfo module_param.ko

filename:       /home/clymber/Studio/kerdevel/module_param/module_param.ko
description:    A Hello, World Module
author:         Clymber Loong
license:        GPL
srcversion:     77F5712DDBB750D3644A994
depends:        
retpoline:      Y
name:           module_param
vermagic:       6.1.28 SMP preempt mod_unload modversions 
parm:           irq:The interrupt line number. (int)
parm:           debug:Debug mode, 1 is on, 0 is off. (int)
parm:           name:Your name (charp)


In [6]:
%%script bash
sudo dmesg -C
sudo insmod ./*.ko name='Clymber' irq=100 debug=1
sudo dmesg -c

[ 8797.163990] Hello, World! name=Clymber, irq=100, debug=1


### 1.4 Verify the result

1. `/proc/modules` shows what kernel modules (drivers) are currently loaded into memory.
2. These module parameters also show up in `sysfs` system.

In [7]:
! grep 'hello' /proc/modules

In [8]:
! ls -l /sys/module/module_param/parameters/

total 0
-rw-rw---- 1 root root 4096 May 20 15:58 debug
-rw-rw---- 1 root root 4096 May 20 15:58 irq
-rw-rw---- 1 root root 4096 May 20 15:58 irqtype
-rw-rw---- 1 root root 4096 May 20 15:58 name


### 1.5 Cleaning up

In [9]:
%%script bash
sudo rmmod ./*.ko
sudo dmesg -c
make clean

[ 8797.441458] Bye Bye, World!
make -C /lib/modules/6.1.28/build M=/home/clymber/Studio/kerdevel/module_param clean
make[1]: Entering directory '/home/clymber/Studio/@linux-6.1.28'
  CLEAN   /home/clymber/Studio/kerdevel/module_param/Module.symvers
make[1]: Leaving directory '/home/clymber/Studio/@linux-6.1.28'


## 2. Reading, changing and verifying parameters

The file in `/sys/module/<module_name>/parameters` is used to read/write the parameter value while the module is loaded.  

Using the file in `sysfs` , if we have permission, we can set any value we want. If for example the irq can only be in the range 1-32 , there is no way to validate it because the user access the variable directly. To solve this problem we can implement an interface for setting an getting parameter values – `kernel_param_ops`:
```c
struct kernel_param_ops {
	/* How the ops should behave */
	unsigned int flags;
	/* Returns 0, or -errno.  arg is in kp->arg. */
	int (*set)(const char *val, const struct kernel_param *kp);
	/* Returns length written or -errno.  Buffer is 4k (ie. be short!) */
	int (*get)(char *buffer, const struct kernel_param *kp);
	/* Optional function to free kp->arg when module unloaded. */
	void (*free)(void *arg);
};
```

### 2.1 Defining `get` and `set` handlers

We want a parameter for interrupt mode to be one from level, edge, polling. In the mean time, we want to save it as a number (1-level, 2-edge, 3-polling).

In [10]:
%%writefile hello-rwv-param.c
#include <linux/module.h>
#include <linux/kernel.h>
#include <linux/moduleparam.h>

enum irq_type {
    IRQ_TYPE_LEVEL,
    IRQ_TYPE_EDGE,
    IRQ_TYPE_POLLING
};

static int irq_type = IRQ_TYPE_LEVEL; // default

Overwriting hello-rwv-param.c


We need to define the `set` function to convert from string to int and the `get` function to convert from int to string.

In [11]:
%%writefile -a hello-rwv-param.c

#define TYPE_BUFF_MAX 16
static int irq_type_write(const char *val, const struct kernel_param *kp)
{
    char type[TYPE_BUFF_MAX];
    char *s;
    
    strncpy(type, val, TYPE_BUFF_MAX);
    type[TYPE_BUFF_MAX-1] = '\0';
    s = strstrip(type);
    
    if (strcmp(s, "level") == 0) {
        irq_type = IRQ_TYPE_LEVEL;
    }
    else if (strcmp(s, "edge") == 0) {
        irq_type = IRQ_TYPE_EDGE;
    }
    else if (strcmp(s, "polling") == 0) {
        irq_type = IRQ_TYPE_POLLING;
    }
    else {
        return -EINVAL;
    }
    return 0;
}

static int irq_type_read(char * buffer, const struct kernel_param *kp)
{
    switch (irq_type) {
    case IRQ_TYPE_LEVEL:
        strcpy(buffer, "Level");
        break;

    case IRQ_TYPE_EDGE:
        strcpy(buffer, "Edge");
        break;

    case IRQ_TYPE_POLLING:
        strcpy(buffer, "Polling");
        break;

    default:
        strcpy(buffer, "error");
        break;
    }

    return strlen(buffer);
}

static const struct kernel_param_ops irqtype_ops = {
    .set = irq_type_write,
    .get = irq_type_read
};

Appending to hello-rwv-param.c


### 2.2 Defining parameters with the handlers registering

In [12]:
%%writefile -a hello-rwv-param.c

module_param_cb(irqtype, &irqtype_ops, NULL, 0660);

Appending to hello-rwv-param.c


In [13]:
! make

make -C /lib/modules/6.1.28/build M=/home/clymber/Studio/kerdevel/module_param modules
make[1]: Entering directory '/home/clymber/Studio/@linux-6.1.28'
  CC [M]  /home/clymber/Studio/kerdevel/module_param/hello-parameters.o
  CC [M]  /home/clymber/Studio/kerdevel/module_param/hello-rwv-param.o
  LD [M]  /home/clymber/Studio/kerdevel/module_param/module_param.o
  MODPOST /home/clymber/Studio/kerdevel/module_param/Module.symvers
  CC [M]  /home/clymber/Studio/kerdevel/module_param/module_param.mod.o
  LD [M]  /home/clymber/Studio/kerdevel/module_param/module_param.ko
make[1]: Leaving directory '/home/clymber/Studio/@linux-6.1.28'


In [14]:
! sudo insmod ./*.ko name='Clymber' irq=100 debug=1 irqtype='edge' && sudo dmesg -c

[ 8799.208301] Hello, World! name=Clymber, irq=100, debug=1


In [15]:
! sudo ls -lh /sys/module/module_param/parameters/*

-rw-rw---- 1 root root 4.0K May 20 15:58 /sys/module/module_param/parameters/debug
-rw-rw---- 1 root root 4.0K May 20 15:58 /sys/module/module_param/parameters/irq
-rw-rw---- 1 root root 4.0K May 20 15:58 /sys/module/module_param/parameters/irqtype
-rw-rw---- 1 root root 4.0K May 20 15:58 /sys/module/module_param/parameters/name


In [16]:
! sudo cat /sys/module/module_param/parameters/irqtype

Edge

In [17]:
%%script bash
sudo su
echo 'polling' >  /sys/module/module_param/parameters/irqtype
cat /sys/module/module_param/parameters/irqtype
exit

Polling

In [18]:
%%script bash
sudo su
echo 'nonsence' >  /sys/module/module_param/parameters/irqtype
cat /sys/module/module_param/parameters/irqtype
exit

bash: line 1: echo: write error: Invalid argument


Polling

In [19]:
%%script bash
sudo rmmod ./*.ko
sudo dmesg -c
make clean

[ 8799.685029] Bye Bye, World!
make -C /lib/modules/6.1.28/build M=/home/clymber/Studio/kerdevel/module_param clean
make[1]: Entering directory '/home/clymber/Studio/@linux-6.1.28'
  CLEAN   /home/clymber/Studio/kerdevel/module_param/Module.symvers
make[1]: Leaving directory '/home/clymber/Studio/@linux-6.1.28'
